In [1]:
import stim
import numpy as np
from numpy.linalg import matrix_power, matrix_rank
import matplotlib.pyplot as plt
import itertools
import pandas as pd
from mec import make_circle

In [321]:

# code = [15,3,11,0,1,0,13,2]
# code = [15,3,11,2,0,0,11,13]
# code = [15,3,12,2,1,1,3,7]
code = [3,15,11,2,0,0,11,13,3,15,2,5]


In [322]:
def cyclic_shift_matrix(l):
    arr = np.eye(l, dtype=int)
    return np.roll(arr, axis=1, shift=1)

ell = code[1]
m = code[0]

x = np.kron(cyclic_shift_matrix(ell), np.eye(m))
y = np.kron(np.eye(ell), cyclic_shift_matrix(m))

A1 = matrix_power(x, code[2])
A2 = matrix_power(y, code[3])
A3 = matrix_power(y, code[4])
A = ( A1 + A2 + A3 ) % 2

B1 = matrix_power(y, code[5])
B2 = matrix_power(x, code[6])
B3 = matrix_power(x, code[7])
B = ( B1 + B2 + B3 ) % 2

Hx = np.hstack([A, B]).astype(int)
Hz = np.hstack([B.T, A.T]).astype(int)

In [362]:
def embed_code(code, init):
    emb_m, emb_ell, A_ind, B_ind = code

    lattice = np.empty((2*emb_m, 2*emb_ell), dtype=object)
    lattice[0][0] = f"x{init}"

    # As = [[A1, A2.T], [A2, A3.T], [A1, A3.T]]
    # Bs = [[B1, B2.T], [B2, B3.T], [B1, B3.T]]
    As = [[A1, A2.T], [A2, A1.T], [A2, A3.T], [A3, A2.T], [A1, A3.T], [A3, A1.T]]
    Bs = [[B1, B2.T], [B2, B1.T], [B2, B3.T], [B3, B2.T], [B1, B3.T], [B3, B1.T]]

    def get_nbr(i, j):
        if (i % 2 == 0):
            if (j % 2 == 0):
                return "x"
            else:
                return "r"
        else:
            if (j % 2 == 0):
                return "l"
            else:
                return "z"

    for i in range(2*emb_m - 1):
        for j in range(2*emb_ell):
            curr_ind = int(lattice[i][j][1:])

            if (i % 2 == 0):
                tmp_A = As[A_ind][1]
            else:
                tmp_A = As[A_ind][0]
            if (j % 2 == 0):
                tmp_B = Bs[B_ind][1]
            else:
                tmp_B = Bs[B_ind][0]

            lattice[(i+1)%(2*emb_m)][j] = f"{get_nbr((i+1)%(2*emb_m), j)}{np.where(tmp_A @ np.eye(m*ell)[curr_ind])[0][0]}"
            lattice[i][(j+1)%(2*emb_ell)] = f"{get_nbr(i, (j+1)%(2*emb_ell))}{np.where(tmp_B @ np.eye(m*ell)[curr_ind])[0][0]}"

    for i in range(2*emb_m):
        for j in range(2*emb_ell):
            if (lattice[i][j][0] == "z"):
                lattice[i][j] = f"z{int(lattice[i][j][1:]) + m*ell}"
            elif (lattice[i][j][0] == "r"):
                lattice[i][j] = f"r{int(lattice[i][j][1:]) + m*ell}"

    return lattice

np.set_printoptions(linewidth=90)
# lattice = embed_code((3,15,2,5), 0)
lattice = embed_code((code[8],code[9],code[10],code[11]), 0)

In [324]:
all_qbts = {}

qbts = np.array([None for i in range(2*m*ell)])
for i in range(lattice.shape[0]):
    for j in range(lattice.shape[1]):
        if lattice[i][j][0] == "r" or lattice[i][j][0] == "l":
            all_qbts[(i,j)] = int(lattice[i][j][1:])
            qbts[int(lattice[i][j][1:])] = (i, j)
x_checks = np.array([None for i in range(m*ell)])
z_checks = np.array([None for i in range(m*ell)])

for i in range(lattice.shape[0]):
    for j in range(lattice.shape[1]):
        if lattice[i][j][0] == "x":
            all_qbts[(i,j)] = int(lattice[i][j][1:]) + 2*m*ell
            x_checks[int(lattice[i][j][1:])] = (i, j)
        elif lattice[i][j][0] == "z":
            all_qbts[(i,j)] = int(lattice[i][j][1:]) + 2*m*ell
            z_checks[int(lattice[i][j][1:])-(m*ell)] = (i, j)

x_rs = []
z_rs = []
for i in range(m*ell):
    gen_qbts = qbts[np.where(Hx[i])[0]]
    x_rs.append(make_circle(gen_qbts)[2])
for i in range(m*ell):
    gen_qbts = qbts[np.where(Hz[i])[0]]
    z_rs.append(make_circle(gen_qbts)[2])

lr_x_checks = []
mr_x_checks = []
sr_x_checks = []
lr_z_checks = []
mr_z_checks = []
sr_z_checks = []


for i, x_check in enumerate(x_checks):
    gen_qbts = qbts[np.where(Hx[i])[0]]

    nonlocal_qbts = []
    if (x_rs[i] > (min(x_rs)+np.std(x_rs))):
        lr_x_checks.append(i)
        continue
    for qbt in gen_qbts:
        if (abs(qbt[0] - x_check[0]) + abs(qbt[1] - x_check[1])) > 1:
            nonlocal_qbts.append(qbt)
    if (len(nonlocal_qbts) == 3):
        mr_x_checks.append(i)
    else:
        sr_x_checks.append(i)

for i, z_check in enumerate(z_checks):
    gen_qbts = qbts[np.where(Hz[i])[0]]

    nonlocal_qbts = []
    if (z_rs[i] > min(z_rs)+np.std(z_rs)):
        lr_z_checks.append(i)
        continue
    for qbt in gen_qbts:
        if (abs(qbt[0] - z_check[0]) + abs(qbt[1] - z_check[1])) > 1:
            nonlocal_qbts.append(qbt)
    if (len(nonlocal_qbts) == 3):
        mr_z_checks.append(i)
    else:
        sr_z_checks.append(i)

In [325]:
for i, x in enumerate(sr_x_checks+mr_x_checks):
    x_check = x_checks[x]
    gen_qbts = qbts[np.where(Hx[x])[0]]

    nonlocal_qbts = []
    for qbt in gen_qbts:
        if (abs(qbt[0] - x_check[0]) + abs(qbt[1] - x_check[1])) > 1:
            nonlocal_qbts.append(qbt)
    print([nonlocal_qbts, x_checks[x]], end=", ")
for i, z in enumerate(sr_z_checks+mr_z_checks):
    z_check = z_checks[z]
    gen_qbts = qbts[np.where(Hz[z])[0]]

    nonlocal_qbts = []
    for qbt in gen_qbts:
        if (abs(qbt[0] - z_check[0]) + abs(qbt[1] - z_check[1])) > 1:
            nonlocal_qbts.append(qbt)
    print([nonlocal_qbts, z_checks[z]], end=", ")

[[(3, 12), (2, 13)], (2, 16)], [[(5, 12), (4, 13)], (4, 16)], [[(3, 14), (2, 15)], (2, 18)], [[(5, 14), (4, 15)], (4, 18)], [[(3, 0), (2, 1)], (2, 4)], [[(5, 0), (4, 1)], (4, 4)], [[(3, 16), (2, 17)], (2, 20)], [[(5, 16), (4, 17)], (4, 20)], [[(3, 2), (2, 3)], (2, 6)], [[(5, 2), (4, 3)], (4, 6)], [[(3, 18), (2, 19)], (2, 22)], [[(5, 18), (4, 19)], (4, 22)], [[(3, 4), (2, 5)], (2, 8)], [[(5, 4), (4, 5)], (4, 8)], [[(3, 20), (2, 21)], (2, 24)], [[(5, 20), (4, 21)], (4, 24)], [[(3, 6), (2, 7)], (2, 10)], [[(5, 6), (4, 7)], (4, 10)], [[(3, 22), (2, 23)], (2, 26)], [[(5, 22), (4, 23)], (4, 26)], [[(3, 8), (2, 9)], (2, 12)], [[(5, 8), (4, 9)], (4, 12)], [[(3, 24), (2, 25)], (2, 28)], [[(5, 24), (4, 25)], (4, 28)], [[(3, 10), (2, 11)], (2, 14)], [[(5, 10), (4, 11)], (4, 14)], [[(5, 16), (1, 12), (0, 13)], (0, 16)], [[(5, 18), (1, 14), (0, 15)], (0, 18)], [[(1, 0), (5, 4), (0, 1)], (0, 4)], [[(1, 16), (5, 20), (0, 17)], (0, 20)], [[(1, 2), (5, 6), (0, 3)], (0, 6)], [[(1, 18), (5, 22), (0, 19)]

In [468]:
def lr_bell_pair(path):
    size = len(path)
    c = stim.Circuit()

    c.append("H", path[:-1][::2])

    c.append("CNOT", path[:size-(size%2)])
    c.append("TICK")
    c.append("CNOT", path[1:size-1+(size%2)])

    c.append("H", path[:-1][1::2])
    c.append("MR", path[1:-1])

    for i in range(2 - (size%2), size-1, 2):
        c.append("CZ", [stim.target_rec(-i), path[0]])
    for i in range(1 + (size%2), size-1, 2):
        c.append("CX", [stim.target_rec(-i), path[-1]])
    
    return c

def lr_CNOT_bell(control, target, bell_pair):
    c = stim.Circuit()

    c.append("TICK")
    c.append("CNOT", [control, bell_pair[0], bell_pair[1], target])
    c.append("MR", bell_pair[0])
    c.append("MRX", bell_pair[1])
    c.append("CX", [stim.target_rec(-2), target])
    c.append("CZ", [stim.target_rec(-1), control])

    return c

def lr_CNOT_no_bell(paths):
    # path[0] is control, path[-1] is target. Reverse path to get reverse CNOT
    c = stim.Circuit()
    num_paths = len(paths)
    
    for path in paths:
        size = len(path)
        c.append("H", path[:-1][2::2])
    c.append("TICK")

    for path in paths:
        size = len(path)
        c.append("CNOT", path[:size-(size%2)])
        # c.append("DEPOLARIZE2", path[:size-(size%2)], 0.01)
    c.append("TICK")

    for path in paths:
        size = len(path)
        c.append("CNOT", path[1:size-1+(size%2)])
        # c.append("DEPOLARIZE2", path[1:size-1+(size%2)], 0.01)
    c.append("TICK")
    
    for path in paths:
        c.append("H", path[:-1][1::2])
    c.append("TICK")

    for path in paths:
        c.append("MR", path[1:-1])
    c.append("TICK")

    for j, path in enumerate(paths):
        size = len(path)
        for i in range(2 - (size%2), size-1, 2):
            c.append("CZ", [stim.target_rec(-(size-2)*(num_paths-1-j)-i), path[0]])
            # c.append("CZ", [stim.target_rec(-i), path[0]])
        for i in range(1 + (size%2), size-1, 2):
            c.append("CX", [stim.target_rec(-(size-2)*(num_paths-1-j)-i), path[-1]])
            # c.append("CX", [stim.target_rec(-i), path[-1]])
    c.append("TICK")

    return c

In [1066]:
c = stim.Circuit()

for key, value in all_qbts.items():
    c.append("QUBIT_COORDS", value, (key[0],key[1],0))
    c.append("QUBIT_COORDS", value+(4*m*ell), (key[0],key[1],1))

c.append("R", [qbt for qbt in all_qbts.values()])
c.append("R", [qbt+(4*m*ell) for qbt in all_qbts.values()])
c.append("TICK")
c.append("H", [all_qbts[x_checks[x_check]] for x_check in sr_x_checks+mr_x_checks])
c.append("TICK")

# left
for x_check in sr_x_checks+mr_x_checks:
    pos = x_checks[x_check]
    if (pos[0] < 2*m-1): c.append("CNOT", [all_qbts[x_checks[x_check]], all_qbts[(pos[0]+1, pos[1])]])
for z_check in sr_z_checks+mr_z_checks:
    pos = z_checks[z_check]
    if (pos[0] < 2*m-1): c.append("CNOT", [all_qbts[(pos[0]+1, pos[1])], all_qbts[z_checks[z_check]]])
c.append("TICK")

z_paths = [[(i, z_checks[z][1]) for i in range(2*m)] for z in mr_z_checks]

# NEED TO SWAP UP ANCILLAS. NOT BEGINNING OR END OF PATH
path_qbts = []
direct_CNOTS = []
for path in z_paths:
    path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
    direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
c += lr_CNOT_no_bell(path_qbts)


# down
for x_check in sr_x_checks+mr_x_checks:
    pos = x_checks[x_check]
    if (pos[1] < 2*ell-1): c.append("CNOT", [all_qbts[x_checks[x_check]], all_qbts[(pos[0], pos[1]+1)]])
for z_check in sr_z_checks+mr_z_checks:
    pos = z_checks[z_check]
    if (pos[1] < 2*ell-1): c.append("CNOT", [all_qbts[(pos[0], pos[1]+1)], all_qbts[z_checks[z_check]]])
c.append("TICK")

# up
for x_check in sr_x_checks+mr_x_checks:
    pos = x_checks[x_check]
    if (pos[1] > 0): c.append("CNOT", [all_qbts[x_checks[x_check]], all_qbts[(pos[0], pos[1]-1)]])
for z_check in sr_z_checks+mr_z_checks:
    pos = z_checks[z_check]
    if (pos[1] > 0): c.append("CNOT", [all_qbts[(pos[0], pos[1]-1)], all_qbts[z_checks[z_check]]])
c.append("TICK")

# right
for x_check in sr_x_checks+mr_x_checks:
    pos = x_checks[x_check]
    if (pos[0] > 0): c.append("CNOT", [all_qbts[x_checks[x_check]], all_qbts[(pos[0]-1, pos[1])]])
for z_check in sr_z_checks+mr_z_checks:
    pos = z_checks[z_check]
    if (pos[0] > 0): c.append("CNOT", [all_qbts[(pos[0]-1, pos[1])], all_qbts[z_checks[z_check]]])
c.append("TICK")


x_paths = [[(i, x_checks[x][1]) for i in range(2*m)] for x in mr_x_checks]

# NEED TO SWAP UP ANCILLAS. NOT BEGINNING OR END OF PATH
path_qbts = []
direct_CNOTS = []
for path in x_paths:
    path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
    direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
c += lr_CNOT_no_bell(path_qbts)

In [1067]:
# # this part is also the same for all codes (assuming a rectangle grid essentially)
x_paths = []
z_paths = []
x_paths = [[(i, x_checks[x][1]) for i in range(2*m)] for x in mr_x_checks]
# z_paths = [[(i, z_checks[z][1]) for i in range(2*m)] for z in mr_z_checks]

# NEED TO SWAP UP ANCILLAS. NOT BEGINNING OR END OF PATH
path_qbts = []
direct_CNOTS = []
for path in z_paths+x_paths:
    path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
    direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

In [1068]:
# # # 10,3,15,11,2,0,0,11,13,3,15,2,5,0.005,0.1333333333333333,9501,9331,0.9821071466161456,8,4,0.5530895454860159,2.5495097567963927
# # # SPECIFIC TO THIS CODE

# # # up right qubit for x checks
# x_checks_2 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==2)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(5)]+[(x_checks[x][0]+1, x_checks[x][1]-4)] for x in x_checks_2]

# # # down for z checks
# # z_checks_0 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==0)]
# # z_paths = [[(z_checks[z][0], z_checks[z][1]+i) for i in range(4)][::-1] for z in z_checks_0]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] for i, qbt in enumerate(path)])
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

# # # up right qubit for x checks
# x_checks_0 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==0)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(5)]+[(x_checks[x][0]+1, x_checks[x][1]-4)] for x in x_checks_0]

# # # down for z checks
# # z_checks_1 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==1)]
# # z_paths = [[(z_checks[z][0], z_checks[z][1]+i) for i in range(4)][::-1] for z in z_checks_1]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

# # # up right qubit for x checks
# x_checks_1 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==1)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(5)]+[(x_checks[x][0]+1, x_checks[x][1]-4)] for x in x_checks_1]

# # # down for z checks
# # z_checks_2 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==2)]
# # z_paths = [[(z_checks[z][0], z_checks[z][1]+i) for i in range(4)][::-1] for z in z_checks_2]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

# # # up qubit for x checks
# x_checks_2 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==2)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(4)] for x in x_checks_2]

# # # down left for z checks
# # z_checks_0 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==0)]
# # z_paths = [[(z_checks[z][0]-1, z_checks[z][1]+4)]+[(z_checks[z][0], z_checks[z][1]+i) for i in range(5)][::-1] for z in z_checks_0]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

# # # up qubit for x checks
# x_checks_1 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==1)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(4)] for x in x_checks_1]

# # # down left for z checks
# # z_checks_2 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==2)]
# # z_paths = [[(z_checks[z][0]-1, z_checks[z][1]+4)]+[(z_checks[z][0], z_checks[z][1]+i) for i in range(5)][::-1] for z in z_checks_2]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)


# # # up qubit for x checks
# x_checks_0 = [x for x in sr_x_checks if ((x_checks[x][1]//2)%3==0)]
# x_paths = [[(x_checks[x][0], x_checks[x][1]-i) for i in range(4)] for x in x_checks_0]

# # # down left for z checks
# # z_checks_1 = [z for z in sr_z_checks if ((z_checks[z][1]//2)%3==1)]
# # z_paths = [[(z_checks[z][0]-1, z_checks[z][1]+4)]+[(z_checks[z][0], z_checks[z][1]+i) for i in range(5)][::-1] for z in z_checks_1]

# direct_CNOTS = []
# path_qbts = []
# for path in x_paths+z_paths:
#     path_qbts.append([all_qbts[qbt] + 4*m*ell if 0 < i < len(path)-1 else all_qbts[qbt] for i, qbt in enumerate(path)])
#     direct_CNOTS.extend([path_qbts[-1][0], path_qbts[-1][-1]])

# c.append("CNOT", direct_CNOTS)
# c.append("TICK")
# # c += lr_CNOT_no_bell(path_qbts)

In [1069]:
c.append("H", [all_qbts[x_checks[x_check]] for x_check in sr_x_checks+mr_x_checks])
c.append("TICK")
c.append("R", [qbt+(4*m*ell) for qbt in all_qbts.values()])
c.append("MR", [all_qbts[x_checks[x_check]] for x_check in sr_x_checks+mr_x_checks])
c.append("MR", [all_qbts[z_checks[z_check]] for z_check in sr_z_checks+mr_z_checks])

In [1070]:
for i, z_check in enumerate(sr_z_checks+mr_z_checks):
    coord = z_checks[z_check]
    c.append("DETECTOR", [stim.target_rec(-len(sr_z_checks+mr_z_checks)+i)], (coord[0], coord[1], 0))

In [1071]:
with open("tmp.svg", "w") as f:
    f.write(str(c.without_noise().diagram("timeslice-svg")))

In [1072]:
samples = c.compile_detector_sampler().sample(10).astype(int)
print(samples.shape)
print(samples)

(10, 39)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [1013]:
len(mr_z_checks)

13

In [1002]:
surface_code_circuit = stim.Circuit.generated(
    "surface_code:unrotated_memory_z",
    rounds=1,
    distance=3,)
    # after_clifford_depolarization=0.001,
    # after_reset_flip_probability=0.001,
    # before_measure_flip_probability=0.001,
    # before_round_data_depolarization=0.001)
print(surface_code_circuit)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(1, 0) 1
QUBIT_COORDS(2, 0) 2
QUBIT_COORDS(3, 0) 3
QUBIT_COORDS(4, 0) 4
QUBIT_COORDS(0, 1) 5
QUBIT_COORDS(1, 1) 6
QUBIT_COORDS(2, 1) 7
QUBIT_COORDS(3, 1) 8
QUBIT_COORDS(4, 1) 9
QUBIT_COORDS(0, 2) 10
QUBIT_COORDS(1, 2) 11
QUBIT_COORDS(2, 2) 12
QUBIT_COORDS(3, 2) 13
QUBIT_COORDS(4, 2) 14
QUBIT_COORDS(0, 3) 15
QUBIT_COORDS(1, 3) 16
QUBIT_COORDS(2, 3) 17
QUBIT_COORDS(3, 3) 18
QUBIT_COORDS(4, 3) 19
QUBIT_COORDS(0, 4) 20
QUBIT_COORDS(1, 4) 21
QUBIT_COORDS(2, 4) 22
QUBIT_COORDS(3, 4) 23
QUBIT_COORDS(4, 4) 24
R 0 2 4 6 8 10 12 14 16 18 20 22 24 1 3 5 7 9 11 13 15 17 19 21 23
TICK
H 1 3 11 13 21 23
TICK
CX 1 2 11 12 21 22 3 4 13 14 23 24 6 5 16 15 8 7 18 17
TICK
CX 1 6 11 16 3 8 13 18 10 5 20 15 12 7 22 17 14 9 24 19
TICK
CX 11 6 21 16 13 8 23 18 0 5 10 15 2 7 12 17 4 9 14 19
TICK
CX 1 0 11 10 21 20 3 2 13 12 23 22 6 7 16 17 8 9 18 19
TICK
H 1 3 11 13 21 23
TICK
MR 1 3 5 7 9 11 13 15 17 19 21 23
DETECTOR(0, 1, 0) rec[-10]
DETECTOR(0, 3, 0) rec[-5]
DETECTOR(2, 1,

In [972]:
c = stim.Circuit(
"""
QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(1, 0) 1
QUBIT_COORDS(2, 0) 2
QUBIT_COORDS(3, 0) 3
QUBIT_COORDS(4, 0) 4
QUBIT_COORDS(0, 1) 5
QUBIT_COORDS(1, 1) 6
QUBIT_COORDS(2, 1) 7
QUBIT_COORDS(3, 1) 8
QUBIT_COORDS(4, 1) 9
QUBIT_COORDS(0, 2) 10
QUBIT_COORDS(1, 2) 11
QUBIT_COORDS(2, 2) 12
QUBIT_COORDS(3, 2) 13
QUBIT_COORDS(4, 2) 14
QUBIT_COORDS(0, 3) 15
QUBIT_COORDS(1, 3) 16
QUBIT_COORDS(2, 3) 17
QUBIT_COORDS(3, 3) 18
QUBIT_COORDS(4, 3) 19
QUBIT_COORDS(0, 4) 20
QUBIT_COORDS(1, 4) 21
QUBIT_COORDS(2, 4) 22
QUBIT_COORDS(3, 4) 23
QUBIT_COORDS(4, 4) 24
R 0 2 4 6 8 10 12 14 16 18 20 22 24 1 3 5 7 9 11 13 15 17 19 21 23
TICK
H 1 3 11 13 21 23
TICK
CX 1 2 11 12 21 22 3 4 13 14 23 24 6 5 16 15 8 7 18 17
TICK
CX 1 6 11 16 3 8 13 18 10 5 20 15 12 7 22 17 14 9 24 19
TICK
CX 11 6 21 16 13 8 23 18 0 5 10 15 2 7 12 17 4 9 14 19
TICK
CX 1 0 11 10 21 20 3 2 13 12 23 22 6 7 16 17 8 9 18 19
TICK
H 1 3 11 13 21 23
TICK
MR 1 3 5 7 9 11 13 15 17 19 21 23
DETECTOR(0, 1, 0) rec[-10]
DETECTOR(0, 3, 0) rec[-5]
DETECTOR(2, 1, 0) rec[-9]
DETECTOR(2, 3, 0) rec[-4]
DETECTOR(4, 1, 0) rec[-8]
DETECTOR(4, 3, 0) rec[-3]
"""
)

c.compile_detector_sampler().sample(10)

array([[False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False]])

In [973]:
with open("tmp.svg","w") as f:
    f.write(str(c.diagram("timeslice-svg")))